# TP MODELISATION DU CHURN

### Rechargement des données préparées

In [68]:
import os
repertoire = "/home/jupyter/EF-form-py-ML/data"
os.chdir(repertoire)
import pickle
file=open("churn_prepared.pydata","rb")
Xy=pickle.load(file)
file.close()

In [7]:
# données sont centrées réduites
Xy.describe()
Xy.dtypes


churnMoy_state    float64
Account Length    float64
Int'l Plan        float64
VMail Plan        float64
VMail Message     float64
Day Mins          float64
Day Calls         float64
Day Charge        float64
Eve Mins          float64
Eve Calls         float64
Eve Charge        float64
Night Mins        float64
Night Calls       float64
Night Charge      float64
Intl Mins         float64
Intl Calls        float64
Intl Charge       float64
CustServ Calls    float64
area__415         float64
area__510         float64
Churn?               bool
dtype: object

In [70]:
X= Xy.drop('Churn?',axis=1)
y=Xy['Churn?']

### Charger les librairies

In [71]:
import sys
sys.path.append("/home/jupyter/EF-form-py-ML/modules")
sys.path

import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from fonctions_metrics import lift
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression

from sklearn.metrics import classification_report,confusion_matrix,accuracy_score,f1_score,roc_auc_score,roc_curve,auc

import matplotlib.pyplot as plt

In [72]:
# split Apprentissage Test
from sklearn.model_selection import train_test_split 
X_train, X_test, y_train, y_test = train_test_split(X,Xy['Churn?'], test_size=0.4,random_state=123)

### PARAMETRER LE MODELE REG PENALISEE
Choisir ridge (L2) ou lasso (L1) ou elasticnet, avec la classe LogisticRegression
Définir une grille d'hyper param pour C
et utliser GridSearchCV pour la balayer

In [ ]:
## Regression logistique avec pénalité lasso et grid search
#### on cherche par CV le meilleur C (1/alpha) le coef de regularisation

In [73]:
param = [{"C":[0.001,0.005,0.01,0.025,0.05,0.075,0.1]}]
# Instancie le modele LASSO : construction classe
lr=LogisticRegression(penalty='l1',solver='liblinear')
# Grid Search : on indique le modele et la grille de param
modeleLassoCV = GridSearchCV (lr,param,cv = 4,n_jobs=-1,scoring='roc_auc')
modeleLassoCV = modeleLassoCV.fit(X_train,y_train)
modeleLassoCV.best_params_
#{'C': 0.0.5}

{'C': 0.05}

In [74]:
# Je constate que le meilleur modeles est 0.05 donc je le reinstancie 
model_LogitL1 =LogisticRegression(penalty='l1',solver='liblinear',C=0.05)
# appliquer la méthode fit() : apprend sur les données de TRAIN
model_LogitL1.fit(X_train,y_train)

LogisticRegression(C=0.05, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l1',
                   random_state=None, solver='liblinear', tol=0.0001, verbose=0,
                   warm_start=False)

In [76]:
# récupérer les coef
coef=list(model_LogitL1.coef_[0]) # c un array2d, 
len(coef) # il y a 20 
coef

[0.17675018976637946,
 0.0,
 0.46274789799405985,
 -0.2301937156620589,
 0.0,
 0.09845888354394194,
 0.0,
 0.4128113632841629,
 0.20040736653217395,
 0.0,
 0.02649549505863574,
 0.0,
 0.0,
 0.0,
 0.0,
 -0.105609792790601,
 0.09788123415582668,
 0.4731083730768118,
 0.0,
 0.0]

In [75]:
# combien d'élemnts non nuls ?
feature_0= list(map(lambda x: x==0.0, coef)) #print(feature_0)
print(feature_0.count(True)) # 6
del feature_0

# PLus on baisse C, plus de colonnes s'annulent (0)
# print(pd.DataFrame({'Coefficients': list(reg.coef_)}, list(X.columns.values)))

10


In [77]:
# Prediction des probabilités de 1 avec    model_LogitL1.predict_proba(X)
# Colonne 1 signifie la seconde modalité du vecteur soit la proba Churn=1
probas_test = model_LogitL1.predict_proba(X_test)[:,1]
probas_train = model_LogitL1.predict_proba(X_train)[:,1]

In [25]:
# score: accuracy taux de bien classé global au cutoff de 0.5
print(model_LogitL1.score(X_test, y_test))
print(model_LogitL1.score(X_train,y_train))

0.8748125937031485
0.8524262131065533


In [78]:
#AUC
print(roc_auc_score(y_train,probas_train))
print(roc_auc_score(y_test,probas_test))

0.8160062954947865
0.8444328922495273


In [47]:
from fonctions_metrics import lift
#compute lift at 10%#
print(lift(probas_train,X_train,y_train))
print(lift(probas_test,X_test,y_test))

# Le lift à 10% : je prend les 10% plus fortes proba de churn (individus les plus risqués),
# taux de churn / taux de churn moyen 

lift at 10 percent : 3.31
3.31
lift at 10 percent : 4.18
4.18


In [48]:
#compute lift at 5%
print(lift(probas_train,X_train,y_train,p=5))
print(lift(probas_test,X_test,y_test,p=5))

lift at 5 percent : 3.55
3.55
lift at 5 percent : 4.67
4.67


In [79]:
# ENR LES RESULTATS DANS UN DICTIONNAIRE POUR POUVOIR COMPARER
model='Score Lasso'
# métriques (liste de dictionnaires)
metriques = [{'model':model,'AUC_test':round(roc_auc_score(y_test,probas_test),3),'lift at 5':lift(probas_test,X_test,y_test,p=5),'lift at 10':lift(probas_test,X_test,y_test,p=10)}]
metriques

lift at 5 percent : 4.67
lift at 10 percent : 4.18


[{'model': 'Score Lasso',
  'AUC_test': 0.844,
  'lift at 5': 4.67,
  'lift at 10': 4.18}]

In [31]:
# mat de confusion à un cutoff donné
pred = (probas_test > 0.14).astype(int)
print(confusion_matrix(y_test, pred))
# parmi les vrais 1 82% sont bien prédits
print(pd.crosstab(y_test,pred).apply(lambda r: r/r.sum(), axis=1))
# parmi les predits 1 34% sont des vrais positifs
print(pd.crosstab(y_test,pred).apply(lambda r: r/r.sum(), axis=0))

target_names = ['Fidèles','Churners']

# métriques au cutoff donné
print(classification_report(y_test,pred, target_names=target_names))
# métriques au cutoff par défaut
print(classification_report(y_test,model_LogitL1.predict(X_test), target_names=target_names))



[[796 354]
 [ 29 155]]
col_0          0         1
Churn?                    
False   0.692174  0.307826
True    0.157609  0.842391
col_0          0         1
Churn?                    
False   0.964848  0.695481
True    0.035152  0.304519
              precision    recall  f1-score   support

     Fidèles       0.96      0.69      0.81      1150
    Churners       0.30      0.84      0.45       184

    accuracy                           0.71      1334
   macro avg       0.63      0.77      0.63      1334
weighted avg       0.87      0.71      0.76      1334

              precision    recall  f1-score   support

     Fidèles       0.88      0.99      0.93      1150
    Churners       0.68      0.17      0.28       184

    accuracy                           0.87      1334
   macro avg       0.78      0.58      0.60      1334
weighted avg       0.85      0.87      0.84      1334



### PARAMETRER LE MODELE REG PENALISEE ELASTICNET

In [80]:
#### Régression logistique elastic net : SGDClassifier : loss = log --> logistique
#### penalty = "elasticnet"
#### alpha entre 1 et 1000 , et l1_ratio entre 0 et 1
from sklearn.linear_model import SGDClassifier
#Defaults to ‘hinge’, which gives a linear SVM. The ‘log’ loss gives logistic regression, a probabilistic classifier. 
#l1_ratio : float
#The Elastic Net mixing parameter, with 0 <= l1_ratio <= 1. l1_ratio=0 corresponds to L2 penalty, l1_ratio=1 to L1. Defaults to 0.15

param = [{"alpha":(0.001,0.01,0.015,0.025,0.004,0.05,0.075,0.1,0.2)} ]
sgdElasticAUCnow = GridSearchCV (SGDClassifier(loss="log", penalty="elasticnet",l1_ratio=0.25,class_weight=None), param, cv = 4,n_jobs=4,scoring='roc_auc')

cvelasticnet = sgdElasticAUCnow.fit(X_train,y_train)
cvelasticnet.best_params_



{'alpha': 0.015}

In [81]:
elasticnet = SGDClassifier(loss="log", penalty="elasticnet", alpha=0.015,l1_ratio=0.25,class_weight=None)
# Apprentissage du modele
elasticnet.fit(X_train,y_train)
elasticnet.coef_
  #compute lift at 10%#

array([[ 0.2345409 ,  0.0348406 ,  0.49181115, -0.29243987,  0.        ,
         0.29740455,  0.        ,  0.2976106 ,  0.15599409,  0.04143732,
         0.1556868 ,  0.02746298,  0.        ,  0.02669302,  0.07585822,
        -0.1731572 ,  0.0782818 ,  0.52338825,  0.        ,  0.        ]])

### Analyser les résultats et les performances


In [82]:
# Prediction des probabilités de 1 avec    model_LogitL1.predict_proba(X)
probas_test = elasticnet.predict_proba(X_test)[:,1]
probas_train = elasticnet.predict_proba(X_train)[:,1]

In [83]:
#AUC
print(roc_auc_score(y_train,probas_train))
print(roc_auc_score(y_test,probas_test))

0.8151072201455833
0.8438468809073724


In [84]:
# AJOUTE LES RESULTATS
model='Score elasticnet'
# métriques (liste de dictionnaires)
metriques.append([{'model':model,'AUC_test':round(roc_auc_score(y_test,probas_test),3),'lift at 5':lift(probas_test,X_test,y_test,p=5),'lift at 10':lift(probas_test,X_test,y_test,p=10)}])
metriques

lift at 5 percent : 4.78
lift at 10 percent : 4.13


[{'model': 'Score Lasso',
  'AUC_test': 0.844,
  'lift at 5': 4.67,
  'lift at 10': 4.18},
 [{'model': 'Score elasticnet',
   'AUC_test': 0.844,
   'lift at 5': 4.78,
   'lift at 10': 4.13}]]

### PARAMETRER LE MODELE RANDOMFOREST
Tester un modle light peu profond et un modele complexe

In [85]:
# exemple randomforest avec des rbres peu profonds
rf_light = RandomForestClassifier(     n_estimators = 300,
                                       criterion = "gini",
                                       max_features = "sqrt",
                                       max_depth = 4,
                                       min_samples_split = 40,
                                       min_samples_leaf = 30,                                                                           
                                       bootstrap = True,
                                       oob_score = True,
                                       n_jobs = -1, # coeurs
                                       random_state = 1234,
                                       class_weight="balanced"
                                     )    


In [86]:
# avec des arbres profonds et dont les feuilles avec 5 ind min
# on bloque à 100 feuilles max
rf_complex= RandomForestClassifier(     n_estimators = 300,
                                       criterion = "gini",
                                       max_features = "sqrt",
                                       max_depth = 8,
                                       min_samples_split = 10,
                                       min_samples_leaf = 5,                                                                           
                                       max_leaf_nodes = 100,
                                       bootstrap = True,
                                       oob_score = True,
                                       n_jobs = -1, # coeurs
                                       random_state = 1234,
                                       class_weight="balanced"
                                     )    

In [87]:
# Apprentissage du modele avec methode fit
rf_light.fit(X_train,y_train)
rf_complex.fit(X_train,y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight='balanced',
                       criterion='gini', max_depth=8, max_features='sqrt',
                       max_leaf_nodes=100, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=5, min_samples_split=10,
                       min_weight_fraction_leaf=0.0, n_estimators=300,
                       n_jobs=-1, oob_score=True, random_state=1234, verbose=0,
                       warm_start=False)

### ANALYSER LES RESULTATS

In [88]:
# Importance des variables
df=pd.DataFrame(rf_complex.feature_importances_,X_train.columns.values)
df.columns=['Importance']
df.sort_values(by='Importance',ascending=False)

,Importance
Day Charge,0.162421
CustServ Calls,0.157764
Day Mins,0.147565
Int'l Plan,0.083224
Eve Charge,0.055548
Eve Mins,0.054607
Intl Charge,0.036346
Intl Calls,0.033874
Intl Mins,0.033751
VMail Message,0.032118


In [89]:
# Prediction des probabilités de 1 ( array2d ) avec fonction predict_proba
probas_test = rf_light.predict_proba(X_test)[:,1]
probas_train = rf_light.predict_proba(X_train)[:,1]

In [90]:
#AUC sur train et test avec roc_auc_score
print(roc_auc_score(y_train,probas_train))
print(roc_auc_score(y_test,probas_test))
# un peu de surapprentissage mais peu ce qui est normal car ce sont des arbres peu profonds

0.922242770017706
0.8936814744801512


In [64]:
print(lift(probas_train,X_train,y_train))
print(lift(probas_test,X_test,y_test))

lift at 10 percent : 5.75
5.75
lift at 10 percent : 5.54
5.54


In [65]:
# AJOUTE LES RESULTATS
model='rf light'
# métriques (liste de dictionnaires)
metriques.append([{'model':model,'AUC_test':round(roc_auc_score(y_test,probas_test),3),'lift at 5':lift(probas_test,X_test,y_test,p=5),'lift at 10':lift(probas_test,X_test,y_test,p=10)}])
metriques

lift at 5 percent : 6.41
lift at 10 percent : 5.54


[{'model': 'Score Lasso',
  'AUC_test': 0.844,
  'lift at 5': 4.67,
  'lift at 10': 4.18},
 [{'model': 'Score elasticnet',
   'AUC_test': 0.845,
   'lift at 5': 4.67,
   'lift at 10': 4.13}],
 [{'model': 'rf light',
   'AUC_test': 0.894,
   'lift at 5': 6.41,
   'lift at 10': 5.54}]]

In [91]:
# Prediction des probabilités de 1 ( array2d ) avec fonction predict_proba
probas_test = rf_complex.predict_proba(X_test)[:,1]
probas_train = rf_complex.predict_proba(X_train)[:,1]

In [93]:
#AUC sur train et test avec roc_auc_score
print(roc_auc_score(y_train,probas_train))
print(roc_auc_score(y_test,probas_test))
# ! Surapprentissage ce qui est normal car ce sont des arbres profonds

0.9906925044265198
0.9094092627599244


In [92]:
# AJOUTE LES RESULTATS
model='rf complex'
# métriques (liste de dictionnaires)
metriques.append([{'model':model,'AUC_test':round(roc_auc_score(y_test,probas_test),3),'lift at 5':lift(probas_test,X_test,y_test,p=5),'lift at 10':lift(probas_test,X_test,y_test,p=10)}])
metriques
# conclusion  le modele RF est plus predictif mais moins confiance dans sa capacité à etre robuste à moyen terme

lift at 5 percent : 7.07
lift at 10 percent : 6.58


[{'model': 'Score Lasso',
  'AUC_test': 0.844,
  'lift at 5': 4.67,
  'lift at 10': 4.18},
 [{'model': 'Score elasticnet',
   'AUC_test': 0.844,
   'lift at 5': 4.78,
   'lift at 10': 4.13}],
 [{'model': 'rf complex',
   'AUC_test': 0.909,
   'lift at 5': 7.07,
   'lift at 10': 6.58}]]

### PARAMETRER LE MODELE BOOSTING


In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

In [ ]:
########### sans randomisation (subsample=1.0 = on prend tt l'échantillon)

In [ ]:
gbt_noRand05=GradientBoostingClassifier(loss='deviance', learning_rate=0.05,
                           n_estimators=500,
                           subsample=1.0, min_samples_split=20, min_samples_leaf=10,
                           max_depth=4)

In [ ]:
# Apprentissage du modele
gbt_noRand05.fit(X_train,y_train)

In [ ]:
# étude du nb iterations optimal en affichant la fonction de perte sur lech test

In [ ]:
niter=500
iter = np.arange(niter) + 1
test_deviance = np.zeros((niter,), dtype=np.float64)
# staged_decision_functio : décision fonction à chaque iteration
for i, y_pred in enumerate(gbt_noRand05.staged_decision_function(X_test)):
    # clf.loss_ assumes that y_test[i] in {0, 1}
    test_deviance[i] = gbt_noRand05.loss_(y_test, y_pred)

plt.figure(figsize=(8, 6))
# Erreur sur le test (evolution deviance)
plt.plot(iter,test_deviance,label='Test',color='darkorange')
        # min vers 100 
# Erreur sur apprentissage (evolution deviance)
plt.plot(iter,gbt_noRand05.train_score_,label='Apprentissage',color='navy')    
# Diminution de l'erreur rapport modele precedant (par rapport au oob)
#plt.plot(iter,gbt_noRand05.oob_improvement_)
plt.legend(loc="upper right", fontsize=12)


In [ ]:
# Prediction des probabilités de 1 , array2d
probas_test = gbt_noRand05.predict_proba(X_test)[:,1]
probas_train = gbt_noRand05.predict_proba(X_train)[:,1]
  #AUC
roc_auc_score(y_train,probas_train)
# 1
roc_auc_score(y_test,probas_test)
# AUC 0.91 sur test

#compute lift at 10%#
lift(probas_train,X_train,y_train)
lift(probas_test,X_test,y_test)
# x 7.6
#compute lift at 5%
lift(probas_test,X_test,y_test,p=5)
# 10.36

In [ ]:
########### algo avec randomisation subsample=0.5 et max_feature = racine(nb variable)
###########
gbt_Rand=GradientBoostingClassifier(loss='deviance', learning_rate=0.05,
                           n_estimators=500,
                           subsample=0.5, min_samples_split=30, min_samples_leaf=20,
                           min_weight_fraction_leaf=0.005,
                           max_depth=3,max_leaf_nodes=12,max_features="sqrt")
# Apprentissage du modele
gbt_Rand.fit(X_train,y_train)

In [ ]:
niter=500
iter = np.arange(niter) + 1
test_deviance = np.zeros((niter,), dtype=np.float64)
# staged_decision_functio : décision fonction à chaque iteration
for i, y_pred in enumerate(gbt_Rand.staged_decision_function(X_test)):
    # clf.loss_ assumes that y_test[i] in {0, 1}
    test_deviance[i] = gbt_Rand.loss_(y_test, y_pred)

# negative cumulative sum of oob improvements (améliration de l'erreur)
#out-of-bag (OOB) estimates can be a useful heuristic to estimate the “optimal” number of boosting iterations. OOB estimates are almost identical to cross-validation estimates but they can be computed on-the-fly without the need for repeated model fitting. OOB estimates are only available for Stochastic Gradient Boosting (i.e. subsample < 1.0), the estimates are derived from the improvement in loss based on the examples not included in the bootstrap sample (the so-called out-of-bag examples). The OOB estimator is a pessimistic estimator of the true test loss, but remains a fairly good approximation for a small number of trees.
cumsum = -np.cumsum(gbt_Rand.oob_improvement_)

In [ ]:

plt.figure(figsize=(8, 6))

plt.plot(iter,test_deviance,label='Test',color='darkorange')
plt.plot(iter,gbt_Rand.train_score_,label='Apprentissage',color='navy')    
plt.figure(figsize=(8, 6))
plt.plot(iter,cumsum,label='OOB loss')
plt.figure(figsize=(8, 6))

plt.plot(iter,gbt_Rand.oob_improvement_,label='amélioration loss sur OOB')
        # min vers 100 
# Erreur sur apprentissage (evolution deviance)
plt.plot(iter,gbt_noRand05.train_score_,label='Apprentissage',color='navy')    

 

In [ ]:
       # min vers 100
     #   pred_at_eachstage=enumerate(gbt_Rand.staged_predict(X_testCR),100)
gbt_Rand=GradientBoostingClassifier(loss='deviance', learning_rate=0.05,
                           n_estimators=150,
                           subsample=0.5, min_samples_split=30, min_samples_leaf=20,
                           min_weight_fraction_leaf=0.005,
                           max_depth=3,max_leaf_nodes=12,max_features="sqrt")
# Apprentissage du modele
gbt_Rand.fit(X_train,y_train)


probas_test = gbt_Rand.predict_proba(X_test)[:,1]
probas_train = gbt_Rand.predict_proba(X_train)[:,1]
  #AUC
roc_auc_score(y_train,probas_train)
# 0,979
roc_auc_score(y_test,probas_test)
# AUC 0.89 sur test

#compute lift at 10%#
lift(probas_train,X_train,y_train)
lift(probas_test,X_test,y_test)
# x 7.3
#compute lift at 5%
lift(probas_test,X_test,y_test,p=5)
# 9.1